# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686569


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:03<00:48,  1.73s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:08,  2.55s/it]

Rendering models:  13%|█▎        | 4/30 [00:09<00:55,  2.13s/it]

Rendering models:  17%|█▋        | 5/30 [00:10<00:47,  1.89s/it]

...

Rendering models:  80%|████████  | 24/30 [00:21<00:03,  1.53it/s]

Rendering models:  90%|█████████ | 27/30 [00:21<00:01,  2.08it/s]

Rendering models:  93%|█████████▎| 28/30 [00:21<00:00,  2.45it/s]

Rendering models:  97%|█████████▋| 29/30 [00:22<00:00,  1.77it/s]

not-logged-in-20a275b53abec8db9f41     0.411751
not-logged-in-6cf01ec1c1cb535db647     0.000665
ELAVATED4LIFE                          5.607497
not-logged-in-a9247261537fa1da33fe     9.752564
not-logged-in-f7e2260ed4a9517a3e4a     0.010301
ctiborekskutr                          0.002395
Andy_Carleen                           0.000363
not-logged-in-55b4a6ae32e5c44a64f3     0.007067
not-logged-in-906a7e366b5c45f86727     0.000280
redboy21                              18.146498
sn346808                               0.019476
LeeJoseph                              0.009090
KrisKeogh                              0.003093
not-logged-in-623cb7cd00f5dbcaf5ac     0.000289
mami2019                               0.000665
not-logged-in-8262566d1147ff08203a    19.607829
amidou                                 0.002880
ControlledChaos                        0.000559
KevinKosa                              0.000294
ranyel.a.e.pasimio                     0.000786
not-logged-in-f3a5d106626fa9edf01c     0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())